In [7]:
import glob
import librosa
import librosa.display
import numpy as np
import _pickle as pickle
import pandas as pd

classes = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}


def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
                                              sr=sample_rate).T, axis=0)
    return mfccs, chroma, mel, contrast, tonnetz


target_files = []


def parse_audio_files(path):
    features = np.empty((0, 193))
    for fn in glob.glob(path):
        try:
            mfccs, chroma, mel, contrast, tonnetz = extract_feature(fn)
        except Exception as e:
            print("Error encountered while parsing file: ", fn)
            continue
        ext_features = np.hstack([mfccs, chroma, mel, contrast, tonnetz])
        features = np.vstack([features, ext_features])
        target_files.append(fn)
    return np.array(features)


ts_features = parse_audio_files('new_test_sounds/*.wav')
tr_features = np.array(ts_features, dtype=pd.Series)

filename = 'Ensemble_Model_protocol2.sav'
model = pickle.load(open(filename, 'rb'))

prediction = model.predict(ts_features)

for i, val in enumerate(prediction):
    print("Input File: ", target_files[i], "|", " Predicted Emotion Is:", classes[int(val)])

FileNotFoundError: [Errno 2] No such file or directory: 'Ensemble_Model_protocol2.sav'

In [5]:
import glob
import librosa
import librosa.display
import numpy as np
import _pickle as pickle
from sklearn import svm
# from sklearn import model_selection
from sklearn.ensemble import VotingClassifier
import pandas as pd
from sklearn.grid_search import GridSearchCV


def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
                                              sr=sample_rate).T, axis=0)
    return mfccs, chroma, mel, contrast, tonnetz


def parse_audio_files(path):
    features, labels = np.empty((0, 193)), np.empty(0)
    labels = []
    for fn in glob.glob(path):
        try:
            mfccs, chroma, mel, contrast, tonnetz = extract_feature(fn)
        except Exception as e:
            print("Error encountered while parsing file: ", fn)
            continue
        ext_features = np.hstack([mfccs, chroma, mel, contrast, tonnetz])
        features = np.vstack([features, ext_features])
        labels = np.append(labels, fn.split('/')[2].split('-')[1].split('.')[0])
    return np.array(features), np.array(labels, dtype=np.int)


tr_features, tr_labels = parse_audio_files('./train_sounds/*.wav')

tr_features = np.array(tr_features, dtype=pd.Series)
tr_labels = np.array(tr_labels, dtype=pd.Series)

model1 = svm.SVC(kernel='linear', C=1000, gamma='auto')
model1.fit(X=tr_features.astype(int), y=tr_labels.astype(int))

model2 = svm.SVC(kernel='rbf', C=1000, gamma='auto')
model2.fit(X=tr_features.astype(int), y=tr_labels.astype(int))

seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
estimators = [('linear', model1), ('rbf', model2)]
ensemble = VotingClassifier(estimators)

results = model_selection.cross_val_score(ensemble, tr_features.astype(int), tr_labels.astype(int), cv=kfold)
ensemble.fit(X=tr_features.astype(int), y=tr_labels.astype(int))

filename = '.\Ensemble_Model_protocol2.sav'

pickle.dump(ensemble, open(filename, 'wb'), protocol=2)

print('Model Saved..')

ValueError: Found array with 0 sample(s) (shape=(0, 193)) while a minimum of 1 is required.